In [6]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import sklearn as sk

training = pd.read_csv('setEntrenamientoMP.csv', low_memory = False)
training = training.drop("Unnamed: 0", axis = 1)
test = pd.read_csv('setTesteoMP.csv', low_memory = False)
kaggle = pd.read_csv('setKaggleMP.csv', low_memory = False)

In [7]:
from sklearn.utils import shuffle
from numpy import sort
from matplotlib import pyplot
from xgboost import plot_importance
from sklearn.feature_selection import SelectFromModel
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, r2_score, classification_report, mean_squared_error
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score, KFold
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import BaggingClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import roc_auc_score, roc_curve

In [8]:
training["person"] = training["person"].astype(str)
test["person"] = test["person"].astype(str)
kaggle["person"] = kaggle["person"].astype(str)

In [37]:
training.loc[training['person'] == 'd21b8e6b']

,person,searched products,viewed product,checkout,ad campaign hit,generic listing,brand listing,staticpage,search engine hit,conversion,...,most_used_channel,SO,most_used_device,dayweek_most_events_done,iphone_searchs,samsung_searchs,timeFirstConversion,sessionDuration,cantSessions,label


In [9]:
label_array = np.array(training['label'])
data_array = np.array(training.drop(columns=['label', 'person']))
kaggle_array = np.array(kaggle.drop(columns='person'))

In [12]:
data_array

array([[0.00000000e+00, 3.14000000e+02, 1.05200000e+03, ...,
        0.00000000e+00, 1.45134844e+03, 3.20000000e+01],
       [1.00000000e+00, 1.30000000e+01, 1.47000000e+02, ...,
        1.12023700e+06, 2.79346875e+03, 1.60000000e+01],
       [2.00000000e+00, 1.10000000e+01, 8.00000000e+00, ...,
        4.53700000e+03, 7.61975000e+02, 2.00000000e+00],
       ...,
       [1.33000000e+02, 1.50000000e+01, 7.60000000e+01, ...,
        0.00000000e+00, 8.57833333e+01, 1.00000000e+00],
       [1.34000000e+02, 6.00000000e+00, 5.00000000e+00, ...,
        0.00000000e+00, 1.52320556e+03, 3.00000000e+00],
       [1.35000000e+02, 7.00000000e+00, 1.60000000e+01, ...,
        0.00000000e+00, 2.13166667e+01, 1.00000000e+00]])

In [ ]:
scaler = StandardScaler()
scaled_values = scaler.fit_transform(train)
train.loc[:, :] = scaled_values

In [10]:
training

,person,searched products,viewed product,checkout,ad campaign hit,generic listing,brand listing,staticpage,search engine hit,conversion,...,most_used_channel,SO,most_used_device,dayweek_most_events_done,iphone_searchs,samsung_searchs,timeFirstConversion,sessionDuration,cantSessions,label
0,ad93850f,0.0,20.0,1.0,10.0,7.0,15.0,0.0,7.0,0.0,...,3,0,1,0,0.0,0.0,0.0,2292.106667,5.0,0.0
1,1b9f7cf6,8.0,9.0,1.0,0.0,0.0,3.0,0.0,1.0,0.0,...,2,0,1,4,0.0,0.0,0.0,2604.350000,2.0,0.0
2,de8fe91b,13.0,27.0,1.0,0.0,3.0,3.0,0.0,4.0,0.0,...,2,11,0,0,0.0,0.0,0.0,2820.708333,2.0,0.0
3,45baf068,0.0,6.0,1.0,5.0,4.0,3.0,0.0,0.0,0.0,...,3,0,1,0,0.0,0.0,0.0,14.866667,1.0,0.0
4,99abca5a,5.0,617.0,17.0,12.0,59.0,29.0,2.0,1.0,2.0,...,0,11,0,5,1.0,0.0,1989594.0,1742.660284,47.0,0.0
5,64f45e8d,1.0,32.0,2.0,1.0,5.0,1.0,1.0,1.0,0.0,...,0,6,0,4,0.0,0.0,0.0,116.666667,2.0,0.0
6,ade6b3bf,5.0,92.0,1.0,13.0,7.0,3.0,1.0,4.0,0.0,...,3,0,1,4,0.0,0.0,0.0,2144.835000,10.0,0.0
7,d73ae9ef,0.0,5.0,1.0,0.0,6.0,4.0,0.0,0.0,0.0,...,0,0,1,0,0.0,0.0,0.0,9.566667,1.0,0.0
8,b2cde41a,0.0,27.0,1.0,1.0,4.0,0.0,0.0,0.0,0.0,...,0,11,0,0,0.0,0.0,0.0,3365.144444,3.0,0.0
9,15ea8012,6.0,23.0,4.0,7.0,12.0,0.0,8.0,7.0,5.0,...,3,0,1,0,0.0,0.0,29883.0,655.233333,1.0,0.0


In [11]:
x_train, x_test, y_train, y_test = train_test_split(data_array, label_array, test_size = 0.33, random_state = 1)

In [16]:
logReg = LogisticRegression()
logReg.fit(x_train, y_train)
y_pred = logReg.predict_proba(x_test)
scoreLogReg = roc_auc_score(y_test, y_pred[:,1])

/home/martin/miniconda3/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


In [17]:
scoreLogReg

0.5740155812061607

In [18]:
knn = KNeighborsClassifier()
knn.fit(x_train, y_train)
y_pred = knn.predict_proba(x_test)
scoreKNN = roc_auc_score(y_test, y_pred[:,1])

In [19]:
scoreKNN

0.5439359608221779

In [20]:
dt = DecisionTreeClassifier()
dt.fit(x_train, y_train)
y_pred = dt.predict_proba(x_test)
scoreDT = roc_auc_score(y_test, y_pred[:,1])

In [21]:
scoreDT

0.5575246357403822

In [22]:
rf = RandomForestClassifier()
rf.fit(x_train, y_train)
y_pred = rf.predict_proba(x_test)
scoreRF = roc_auc_score(y_test, y_pred[:,1])

/home/martin/miniconda3/lib/python3.6/site-packages/sklearn/ensemble/forest.py:248: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


In [23]:
scoreRF

0.6981530809568248

In [24]:
ada = AdaBoostClassifier()
ada.fit(x_train, y_train)
y_pred = ada.predict_proba(x_test)
scoreAda = roc_auc_score(y_test, y_pred[:,1])

In [25]:
scoreAda

0.8085477218240933

In [26]:
xgb = XGBClassifier()
xgb.fit(x_train, y_train)
y_pred = xgb.predict_proba(x_test)
scoreXgb = roc_auc_score(y_test, y_pred[:,1])

In [27]:
scoreXgb

0.821799995363699

In [36]:
kaggle

,Unnamed: 0,person,searched products,viewed product,checkout,ad campaign hit,generic listing,brand listing,staticpage,search engine hit,...,most_viewed_condition,most_used_channel,SO,most_used_device,dayweek_most_events_done,iphone_searchs,samsung_searchs,timeFirstConversion,sessionDuration,cantSessions
0,0,4886f805,1.0,4.0,1.0,0.0,1.0,0.0,0.0,1.0,...,2.0,2.0,0.0,1.0,0.0,0.0,0.0,0.0,23.133333,1.0
1,1,0297fc1e,6.0,404.0,7.0,29.0,21.0,4.0,0.0,0.0,...,0.0,0.0,0.0,1.0,1.0,4.0,0.0,0.0,2241.501498,89.0
2,2,2d681dd8,1.0,13.0,1.0,1.0,1.0,5.0,0.0,2.0,...,0.0,3.0,11.0,0.0,3.0,0.0,0.0,0.0,7067.166667,2.0
3,3,cccea85e,1.0,708.0,1.0,15.0,20.0,7.0,5.0,26.0,...,3.0,2.0,11.0,0.0,1.0,0.0,0.0,0.0,2840.459722,12.0
4,4,4c8a8b93,9.0,177.0,2.0,14.0,14.0,8.0,0.0,13.0,...,2.0,4.0,0.0,1.0,0.0,0.0,0.0,0.0,640.896296,9.0
5,5,29ebb414,11.0,8.0,1.0,4.0,3.0,0.0,0.0,4.0,...,2.0,3.0,0.0,1.0,0.0,9.0,0.0,0.0,1300.450000,3.0
6,6,3dc1950f,2.0,545.0,2.0,3.0,24.0,6.0,1.0,7.0,...,3.0,0.0,11.0,0.0,4.0,0.0,0.0,0.0,3126.200000,19.0
7,7,8ea4c165,17.0,97.0,3.0,13.0,8.0,5.0,0.0,9.0,...,2.0,3.0,11.0,0.0,5.0,0.0,0.0,0.0,4296.104762,7.0
8,8,d8cfe234,5.0,23.0,2.0,0.0,1.0,2.0,0.0,2.0,...,0.0,0.0,11.0,0.0,1.0,0.0,4.0,0.0,14773.129167,4.0
9,9,d6bc64df,9.0,116.0,1.0,7.0,9.0,0.0,0.0,5.0,...,0.0,3.0,0.0,0.0,2.0,5.0,0.0,0.0,7375.155556,6.0


In [28]:
kaggle_predict_Ada = ada.predict_proba(kaggle_array)

ValueError: Input contains NaN, infinity or a value too large for dtype('float32').

In [ ]:
kaggle_submit_Ada = {'person': kaggle["person"], 'label': kaggle_predict_Ada[:,1]}
kaggle_submit_AdaDF = pd.DataFrame(data=kaggle_submit_Ada)
kaggle_submit_AdaDF